# Mineração de Dados Massivos
## Analise de descrição de mercadorias de Notas Fiscais Eletrônicas

# Ambiente

## Bibliotecas

In [ ]:
# Iniciando Ambiente
import numpy as np, pandas as pd, time, random
import datetime as dt
import re 

import nltk
from nltk.corpus import stopwords
import unicodedata


# Se necessário, descomente as linhas a seguir para ler as stopwords
nltk.download('stopwords')

from sklearn.feature_extraction.text import CountVectorizer

## Módulos

In [ ]:
# Módulos de código de programação
#!pip install import-ipynb
import import_ipynb
# Carga de Arquivos
from carga import doCarga
# Tratamento de Dados         
from tratamento import doTratamento
# Processamento Naive Bayers
#from naivebayers import doNaiveBayers

from MLPClassifier import *
from SGDClassifier import *

In [ ]:
# Contador de tempo
def tic():
    global _start_time 
    _start_time = time.time()

def tac():
    t_sec = round(time.time() - _start_time)
    (t_min, t_sec) = divmod(t_sec,60)
    (t_hour,t_min) = divmod(t_min,60) 
    print('Duração: {}hour:{}min:{}sec'.format(t_hour,t_min,t_sec))

## Variáveis de Ambiente

In [ ]:
# Pasta aonde estão os dados
diretorio = '..\\dados\\'
#diretorio = "F:\\Weisner\\Documentos\\MEGA Estudos\\UNB - MDM - Mineração de Dados Massivos\\Artigo\\" 


# Abre arquivo de dados

In [ ]:
# Carrega os dados
dfNotas = doCarga(diretorio, opcao=3)
print ("{} registros carregados".format(dfNotas.size))

# Processa se necessário

In [ ]:
## Chama processamento manualmente

#df_original = doTratamento(df_original)
#df_original.NovaDescricao
#df_original.info()
#tac()
dfNotas

## Amostsra dos Dados processados

In [ ]:
%matplotlib inline
dfNotas['Categoria'].value_counts().plot.bar()

# Naive Bayers

In [ ]:
#print (doNaiveBayers(dfNotas))
dfNotas

# Análise

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import warnings
from pylab import rcParams
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split


In [ ]:
%matplotlib inline
rcParams['figure.figsize'] = 10,6
#warnings.filterwarnings('ignore')
sns.set(style='darkgrid')

# Preleção
Preparação dos datasets utilizado em todos algorítimos

In [ ]:
# Separa os contadores
count_class_0, count_class_1, count_class_2, count_class_3, count_class_4 = dfNotas['Categoria'].value_counts()

In [ ]:
# Divide datasets por classes

df_class_0 = dfNotas[dfNotas['Categoria'] == "31.0simsim"]
df_class_1 = dfNotas[dfNotas['Categoria'] == "38.0simsim"]
df_class_2 = dfNotas[dfNotas['Categoria'] == "38.0simnão"] 
df_class_3 = dfNotas[dfNotas['Categoria'] == "30.0simsim"]         # Tamanho escolhido
df_class_4 = dfNotas[dfNotas['Categoria'] == "31.0nãonão"]         # Classse MUITO pequena - UNDERSAMPLING

# Equipara os tamanhos 30.0simsim que tem 3090 ocorrências
# Foram equiparados ao tamanho da classe 3
MaxSize = df_class_3.size 
df_class_0 = df_class_0.sample(MaxSize)
df_class_1 = df_class_1.sample(MaxSize)
df_class_2 = df_class_2.sample(MaxSize)

# Random Oversampling
df_class_4 = df_class_4.sample(MaxSize, replace=True)

# Agrupa novamente
dfAjustado = pd.concat([df_class_0, df_class_1, df_class_2, df_class_3, df_class_4], axis=0)

In [ ]:
#  Separar Dados de Atributos
X = dfAjustado.iloc[:, 1:2].values  # Categoria
y = dfAjustado.iloc[:, 2:3].values  # NovaDescricao

In [ ]:
type(X)

In [ ]:
# Recorta os datasets de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Algorítimos

## Naives

In [ ]:
#W
import sklearn
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

In [ ]:
X_train

In [ ]:
# W

Result = sklearn.model_selection.cross_val_score(text_clf, X_train, y_train, cv=5)

In [ ]:
print (Result)

In [ ]:
# T
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline

# Cria o modelo baseado no Multinomial Naive_Bayes
model = make_pipeline(TfidfTransformer, MultinomialNB)

In [ ]:
#T
# Treina o modelo com o dados de treinamento
result = model.fit(X_train, y_train)

In [ ]:
#J
classificador = nltk.NaiveBayesClassifier.train(X_train)

# Avaliação